In [7]:
import pandas as pd
#load merged.csv
merged = pd.read_csv(r'C:\Users\zuzan\OneDrive\Desktop\Python Load\Buurt_.csv', sep = ',')
merged.buurtcode = merged.buurtcode.astype(str)
merged = merged[['buurtcode', 'buurtname']]
merged.drop_duplicates(subset ='buurtcode', keep = 'first', inplace = True)
print(merged.shape)
print(merged.head())

(13305, 2)
  buurtcode                                    buurtname
0     30000                           Appingedam-Centrum
1     30001                              Appingedam-West
2     30002                              Appingedam-Oost
3     30007  Verspreide huizen Damsterdiep en Eemskanaal
4     30008  Verspreide huizen ten zuiden van Eemskanaal


In [8]:
import pandas as pd
criminality = pd.read_csv(r'C:\Users\zuzan\OneDrive\Desktop\repository\product3team9\data\raw\cbs_datasets\cbs_criminality.csv', 
                     error_bad_lines = False, sep = ';')
criminality = criminality[['NBH_code', 'Total_theft_from_house','Destruction','Violence_and_sexual_abuse']]
criminality.head()

,NBH_code,Total_theft_from_house,Destruction,Violence_and_sexual_abuse
0,NL00,.,.,.
1,GM1680,2,2,3
2,WK168000,1,2,2
3,BU16800000,1,2,2
4,BU16800009,0,0,0


In [9]:
criminality = criminality[criminality['NBH_code'].str.contains('BU')]
criminality['NBH_code'] = criminality['NBH_code'].astype(str)
for a in range(len(criminality)):
    if criminality['NBH_code'].iloc[a][2:3] == '0' and criminality['NBH_code'].iloc[a][3:4] != '0':
        criminality['NBH_code'].iloc[a] = ''.join(criminality['NBH_code'].iloc[a].rsplit('BU0'))
    elif criminality['NBH_code'].iloc[a][2:4] == '00' and criminality['NBH_code'].iloc[a][4:5] != '0':
        criminality['NBH_code'].iloc[a] = ''.join(criminality['NBH_code'].iloc[a].rsplit('BU00'))
    elif criminality['NBH_code'].iloc[a][2:5] == '000':
        criminality['NBH_code'].iloc[a] = ''.join(criminality['NBH_code'].iloc[a].rsplit('BU000'))
    else:
        criminality['NBH_code'].iloc[a] = criminality['NBH_code'].iloc[a][2:]
   
criminality.head(100)

,NBH_code,Total_theft_from_house,Destruction,Violence_and_sexual_abuse
3,16800000,1,2,2
4,16800009,0,0,0
6,16800100,2,0,1
7,16800109,0,6,6
9,16800200,0,3,0
...,...,...,...,...
134,1970302,0,1,1
135,1970303,1,1,2
136,1970304,2,1,1
137,1970305,0,0,0


In [12]:
cleaned_criminality = pd.merge(criminality, merged, left_on='NBH_code', right_on='buurtcode', how = 'left')
cleaned_criminality = cleaned_criminality.loc[cleaned_criminality['buurtname'].isna() == False]
cleaned_criminality = cleaned_criminality[['buurtname', 'Total_theft_from_house','Destruction','Violence_and_sexual_abuse']]
cleaned_criminality.shape

(13305, 4)

In [31]:
cleaned_criminality.isna().sum()

buurtname                      0
Total_theft_from_house       933
Destruction                  923
Violence_and_sexual_abuse    923
dtype: int64

In [32]:
cleaned_criminality = cleaned_criminality.dropna(subset = ['Total_theft_from_house', 'Destruction', 'Violence_and_sexual_abuse'])

In [26]:
cleaned_criminality['Total_theft_from_house']=cleaned_criminality['Total_theft_from_house'].str.extract(r'(\d)')
cleaned_criminality['Destruction']=cleaned_criminality['Destruction'].str.extract(r'(\d)')
cleaned_criminality['Violence_and_sexual_abuse']=cleaned_criminality['Violence_and_sexual_abuse'].str.extract(r'(\d)')

In [33]:
cleaned_criminality['Total_theft_from_house'] = cleaned_criminality['Total_theft_from_house'].astype(int)
cleaned_criminality['Destruction'] = cleaned_criminality['Destruction'].astype(int)
cleaned_criminality['Violence_and_sexual_abuse'] = cleaned_criminality['Violence_and_sexual_abuse'].astype(int)
print(cleaned_criminality.dtypes)

buurtname                    object
Total_theft_from_house        int32
Destruction                   int32
Violence_and_sexual_abuse     int32
dtype: object


<ipython-input-33-52a00ab78e7b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_criminality['Total_theft_from_house'] = cleaned_criminality['Total_theft_from_house'].astype(int)
<ipython-input-33-52a00ab78e7b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_criminality['Destruction'] = cleaned_criminality['Destruction'].astype(int)
<ipython-input-33-52a00ab78e7b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_in

In [34]:
print(cleaned_criminality.dtypes)

buurtname                    object
Total_theft_from_house        int32
Destruction                   int32
Violence_and_sexual_abuse     int32
dtype: object


In [36]:
cleaned_criminality.to_csv('cleaned_criminality.csv', index=False)